In [9]:
import os
import rasterio
import cv2
import numpy as np

In [16]:

def save_flux_map_as_rgb(src, save_dir, address=None, month=None):
    """
    Save a GeoTIFF file containing monthly flux maps as RGB images. Option to save a specified month's flux map 
    or the annual (sum of all channels) flux map.
    
    Args:
    src (nparray): Path to the GeoTIFF file.
    save_dir (str): Directory to save the RGB images.
    month (int, optional): The month to save (1 for January, 12 for December). 
                           If None, the annual flux map (sum of all months) will be saved.
    
    Raises:
    FileNotFoundError: If the file cannot be found or read.
    ValueError: If the month is not between 1 and 12 or if the file does not contain 12 bands.
    """
    if month is not None and (month < 1 or month > 12):
        raise ValueError("Month must be between 1 and 12.")
    
    if month is None:
        # Read all 12 bands and compute the annual flux map
        annual_flux = np.zeros(src.read(1).shape, dtype='float32')
        for i in range(1, 13):
            annual_flux += src.read(i)
        data_to_save = annual_flux
        filename = os.path.join(save_dir, f"annualFlux_{address}.jpg")
    else:
        # Read the specified month's band
        data_to_save = src.read(month)
        filename = os.path.join(save_dir, f"{month}monthlyFlux_{address}.jpg")

    # Normalize the data for saving
    data_normalized = data_to_save.astype('float32')
    max_val = np.percentile(data_to_save, 99)
    data_normalized /= max_val

    # Clip values to the range [0, 1]
    data_normalized = np.clip(data_normalized, 0, 1)

    # Convert to RGB image
    data_rgb = (data_normalized * 255).astype('uint8')
    data_rgb = cv2.applyColorMap(data_rgb, cv2.COLORMAP_VIRIDIS)

    # Save the image
    cv2.imwrite(filename, data_rgb)

In [19]:
data_root = r"D:\solarRec\solarRec\data\test_grid"
save_dir = os.path.join(data_root, "flux_maps")
os.makedirs(save_dir, exist_ok=True)
address_dir = os.path.join(data_root, "addresses")
for address in os.listdir(address_dir):
    flux_map_path = os.path.join(address_dir, address, f"monthlyFlux_{address}.tif")
    # print(os.path.exists(flux_map_path))
    try:
        with rasterio.open(flux_map_path) as src:
            # Check if there are exactly 12 bands
            if src.count != 12:
                raise ValueError("GeoTIFF file does not contain exactly 12 bands (requires 12 bands for each month).")
            
            save_flux_map_as_rgb(src, save_dir=save_dir, address=address, month=None)   # set month to None to get annual flux
    except rasterio.errors.RasterioIOError as e:
        raise FileNotFoundError(f"Unable to locate or read the file at {flux_map_path}") from e
    